# TP6 - _Docking_ do 5-fluoracil ao DNA usando Algoritmos Genéticos

O trabalho da sexta (e última) aula prática é dedicado ao estudo da agregação do 5-fluoracil (5-FU) a um fragmento de DNA. O 5-FU é um fármaco usado no tratemento do cancro, mas é ele prórpio também um agente carcinogénico, podendo interagir com o DNA de células saudáveis. Por outro lado, a interação de heterociclos com fragmentos individuais de DNA tem servido como inspiração para o desenvolvimento de (bio-)nanodispositivos capazes de transmitir energia entre diferentes partes de um circuito molecular. Deste modo, o estudo da agregação  um heterocíclo como o 5-FU ao DNA com diferentes sequências é interessante tanto sob o ponto de vista do estudo da acção carcinogénica do mesmo (e eventual desenvolvimento de variantes estruturais menos carcinogénicas) como sob o ponto de vista de novas formas de processamento da energia.

Este trabalho expõe um protocolo mínimo de _docking_ molecular para encontrar agregados DNA-5-FU com uma molécula de 5-FU e uma cadeia de DNA com 12 resíduos com o padrão repetitivo ACG. A orientação relativa da molécula de 5-FU será optimizada usando algoritmos genéticos, e a energia de cada sistema proposto será levada a cabo usando o campo de forças GFN-FF, o qual inclui efeitos de polarização, e encontra-se implementado no software `xtb`, podendo reconstruir a topologia do sistema a partir das coordenadas dadas num ficheiro em formato `xyz`.



## Preparação do Ambiente Python

Este _notebook_ é usado como protocolo e folha de excercício deste trabalho é uma tecnologia que permite combinar texto formatado com código Python. Existem várias implementações desta tecnologia (ver TP2). No entanto, este trabalho prático requer o uso de um programa externo (`xtb`).  A configuração deste _software_ pode depender do ambiente usado, pelo que este trabalho foi desenhado para **correr apenas na plataforma Google Colab**. A Google Colab é uma implementação de cálculo na _cloud_ desenvolvida pela Google e disponível em https://colab.research.google.com/ . Possui algumas vantagens relativamente ao uso de notebooks da Jupyter: os cálculos correm numa máquina remota da Google, não há nececidade de instalar software no computador local (basta um browser recente) e é gratuito, para o nível de acesso mais básico.

O ambiente oferecido pelo Google Colab ja inclui vários pacotes necessários para cálculo numérico, gráficos e tratamento de dados. Para este trabalho específico, apenas é necessário instalar e configurar o `xtb`, incluíndo definir um par de funções necessário para que este corra a partir dos comandos do python.

In [ ]:
# install py3Dmol
!pip install --user py3Dmol
import sys
sys.path.append('/root/.local/lib/python3.7/site-packages')

# install openbabel (only the software, not the python bindings)
!apt-get install openbabel

# install XTB
!wget -c https://github.com/grimme-lab/xtb/releases/download/v6.5.1/xtb-6.5.1-linux-x86_64.tar.xz
!tar xf xtb-6.5.1-linux-x86_64.tar.xz
!export PATH=/content/xtb-6.5.1/bin:${PATH}

# implementação dos algoritmos geneticos
!wget https://raw.githubusercontent.com/teixeirafilipe/ChemOutReach/main/MM_MQM2223/ga.py

# aditional functions for xtb
import subprocess
def shell(cmd, shell=True):
  "runs a command in the linux shell, adapted from Jimmy Kromann."
  if shell:
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  else:
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  output, err = p.communicate()
  return output

def run_xtb(args):
  return shell(f"ulimit -s unlimited; OMP_STACKSIZE=12G OMP_MAX_ACTIVE_LEVELS=1 /content/xtb-6.5.1/bin/xtb {args}")


## Importar Pacotes e Carregar Funções Utilitárias

Depois de configurar o seu ambiente python, deverá executar a célcula seguinte, de forma a carregar os pacotes necessários (instrução `import`), assim como definir algumas funções, classes e constantes usadas ao longo do trabalho.

In [ ]:
import time
import ga
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import py3Dmol

atomic_symbols=['XX','H','He','Li','Be','B','C','N','O','F','Ne',
                'Na','Mg','Al','Si','P','S','Cl','Ar']

class Molecule():
  def __init__(self,fn=None, name=None):
    self.name='None'
    self.symbols=list()
    self.geo=np.array([])
    if fn:
      self.read_xyz(fn)
    if name:
      self.name=name

  def copy(self):
    "makes a copy of the Molecule"
    o = Molecule()
    o.name = self.name
    o.symbols = self.symbols.copy()
    o.geo = self.geo.copy()
    return o

  def natoms(self):
      return len(self.symbols)
  
  def get_mmff94_energy(self):
    self.write_xyz('Molecule_mmff94.xyz')
    r=shell("obenergy -ff mmff94 Molecule_gfnff.xyz")
    data=r.decode().split('\n')
    for line in data:
      if 'TOTAL ENERGY =' in line:
        return(4.18*float(line.split()[-2]))

  def get_gnfff_energy(self):
    self.write_xyz('Molecule_gfnff.xyz')
    run_xtb("--gfnff Molecule_gfnff.xyz --scc > Molecule.out")
    data=open("Molecule.out",'r').readlines()
    for line in data:
      if '| TOTAL ENERGY  ' in line:
        return(float(line.split()[-3]))
  
  def show(self, label=None): 
    p = py3Dmol.view()
    p.addModel(self.write_xyz())
    #p.setStyle({'stick':{}})
    p.setStyle({'stick':{},'sphere':{'scale':0.3}})
    p.setBackgroundColor('0xeeeeee')
    if label == 'C':
      for i,s in enumerate(self.symbols):
        if s.upper().strip()=='C':
          p.addLabel(f"C{i+1}",{'position': {'x':self.geo[i,0], 'y':self.geo[i,1], 'z':self.geo[i,2]}})
    p.zoomTo()
    p.show()

  def read_xyz(self, fn):
    "Reads a XYZ file"
    with open(fn,'r') as f:
      data = f.readlines()
    natoms = int(data[0])
    g = list()
    for i in range(2,2+natoms):
      l = data[i].split()
      self.symbols.append(l[0].capitalize())
      g.append(list(map(float,l[1:4])))
    self.geo=np.array(g)

  def centre_location(self):
    "returns the location of the geometrical centre as a vector"
    return self.geo.mean(axis=0)

  def shift(self, displacement):
    "Displaces the molecule as a whole, given a 3D vector (np.array)"
    self.geo += displacement

  def center(self):
    "Displaces the molecule so that its geometrical centre is at the origin."
    self.shift(-self.centre_location())

  def write_xyz(self,fn=None, comment=''):
    if comment:
      s = f"{self.natoms()}\n{comment.strip()}\n"
    else:
      s = f"{self.natoms()}\n Created by Molecule Class\n"
    for n in range(self.natoms()):
        s += f"{self.symbols[n]:3s}"
        for i in range(3):
          s += f" {self.geo[n,i]:16.6f}"
        s += '\n'
    if fn:
      with open(fn,'w') as f:
        f.write(s)
    else:
      return s

  def orient(self, x, y, z, theta, phi, gamma, debug=False):
    M = np.array([[ np.cos(phi),  0, np.sin(phi)],
                  [           0,  1, 0],
                  [-np.sin(phi),  0, np.cos(phi)]])
    for n in range(self.natoms()):
      self.geo[n] = np.matmul(M,self.geo[n])
    M = np.array([[ 1, 0, 0],
                  [ 0,  np.cos(theta), np.sin(theta)],
                  [ 0, -np.sin(theta), np.cos(theta)]])
    for n in range(self.natoms()):
      self.geo[n] = np.matmul(M,self.geo[n])
    M = np.array([[ np.cos(gamma), np.sin(gamma), 0],
                  [-np.sin(gamma), np.cos(gamma), 0],
                  [0 , 0, 1]])
    for n in range(self.natoms()):
      self.geo[n] = np.matmul(M,self.geo[n])
    self.shift(np.array([x,y,z]))

  def add_molecule(self,other):
    if not isinstance(other,Molecule):
      raise TypeError("Cannot add to a non-Molecule object.")
    original_natoms = self.natoms()
    added_atoms = other.natoms()
    if original_natoms == 0:
        return other.copy()
    if added_atoms == 0:
        return self.copy()
    o = self.copy()
    for n in range(added_atoms):
      o.symbols.append(other.symbols[n])
    o.geo = np.concatenate((o.geo,other.geo))
    return o

  def __add__(self,other):
    return(self.add_molecule(other))

  def make_distance_matrix(self):
    natoms, _ = self.geo.shape
    o=np.zeros((natoms,natoms))
    for i in range(natoms-1):
        for j in range(i+1,natoms):
            d=np.linalg.norm(self.geo[i,:]-self.geo[j,:])
            o[i,j]=d
            o[j,i]=d
    return o
  
  def check_for_colisions(self,r=1.0):
    natoms, _ = self.geo.shape
    for i in range(natoms-1):
      for j in range(i+1,natoms):
          d=np.linalg.norm(self.geo[i,:]-self.geo[j,:])
          if d<r:
            return True
    return False
    
def plot_trj_energy(fn):
  data=open(fn,'r').readlines()
  e=list()
  for line in data:
    if 'energy:' in line:
      l=line.split()
      for i,token in enumerate(l):
        if token=='energy:':
          e.append(float(l[i+1]))
  e = np.array(e)
  e = 2625.5*(e-e.min())
  fig,ax = plt.subplots()
  ax.scatter(np.arange(1,len(e)+1),e)
  ax.set_xlabel("Passo (n)")
  ax.set_ylabel("Energia Relativa (kJ/mol)")
  plt.show()


## Estudo das Moléculas Individuais

Começamos o trabalho por carregar as moléculas individuais (DNA e 5-FU) e estimar a energia delas. O valor E(DNA) + E(5-FU) é o valor base da energia do sistema, a partir do qual podemos estimar a energia de interacção entre o DNA e o 5-FU para os diversos agregados.

In [ ]:
! wget https://raw.githubusercontent.com/teixeirafilipe/ChemOutReach/main/MM_MQM2223/5fu.xyz
! wget https://raw.githubusercontent.com/teixeirafilipe/ChemOutReach/main/MM_MQM2223/ACG.xyz

# Carregar a Molécula do 5-fu, centrar no espaço, visualizar e calcular a energia
fu = Molecule('5fu.xyz')
fu.center()
fu.show()
print(f"A energia de uma molécula de 5-FU é: {fu.get_gnfff_energy()} Eh")

# Carregar a Molécula do DNA, centrar no espaço, visualizar e calcular a energia
dna = Molecule('ACG.xyz')
dna.center()
dna.show()
print(f"A energia de um fragmento de DNA é: {dna.get_gnfff_energy()} Eh")


## _Docking_ Usando Algoritmos genéticos.

Os estudos de docking presupõem normalmente uma conformação rígida das moléculas. Neste contexto, a molécula do DNA permenecerá imóvel no centro do sistema de coordenadas. Já  a molécula de 5-FU tem seis graus de liberdade possíveis:
* 3 coordenadas espaciais (_xyz_) para o seu centro
* 3 rotações ao longo dos eixos _xx_, _yy_ e _zz_ que permitem orientar a molécula no espaço

Temos, portanto 6 coordenadas para cada molécula, fazendo um total de 60 coordenadas para optimizar. 

Um sistema deste tipo tem múltiplos mínimos locais da energia, pelo que um método simples de optimização de geometria iria convergir para o mínimo local mais próximo, não permitindo a devida exploração do espaço configuracional. Por outro lado, as Dinâmicas Moleculares iriam requerer a hidratação do DNA (a hélice B não é estável em vácuo). Teriam portanto que cobrir não só os processos de agregação das várias moléculas de 5-FU, mas também processos de deslocamento de água. Todos estes processos tomam lugar numa escala de tempo bastante longa, tornando o cálculo computacionalmente difícil. Finalmente, os métodos de energia livre avançados (metadinâmicas, Monte Carlo, Umbrela Sampling) também irão requerer pelo menos o constrangimento da cadeia de DNA. Para além disso, estes métodos iriam requerer um número elevado de iterações de forma a fazer uma boa amostragem da superfífice de energia livre, de tal modo a podermos encontrar algumas configurações interessantes.

Nestes casos, o uso de algoritmos genéticos pode ser uma alternativa viável. A teoria-base dos algoritmos genéticos pode ser resumida em poucas linhas:
* Cada possível solução para o problema encontra-se codificada no genoma de um "Indivíduo"
* Vários "Individuos" existem numa população.
* Em cada iteração, acontecem os seguintes eventos na população:
  * Os indivíduos são ordenados pela excelência do seu genoma (neste caso, quanto mais baixa a energia do sistema, melhor)
  * Os melhores indivídios são escolhidos para reprodução... os restantes morrem para dar lugar a novos individuos (soluções).
  * De entre os "sobreviventes", quanto melhor a qualidade do seu genoma, maiores as suas probabilidades de "acasalar" com outro indivíduo.
  * Selecionamos pares de indivíduos para acasalar... isto traduz-se na re-composição dos seus genomas num novo indivíduo
  * Aquando da criação de um novo indivíduo, pode occorrer uma mutação (alteração aleatória do seu genoma)
  * Os novos individuos são avaliados, e uma nova iteração pode começar


As duas ideias fundamentais por detraís dos Algoritmos Genéticos são:
1. **A solução óptima do problema pode ser aproximada a partir do cruzamento de soluções menos optimizadas**
1. **As mutações que ocorrem numa população são uma oportunidade para o sistema explorar soluções alternativas de adaptação ao meio (soluções do problema)**

No caso do nosso problema de _docking_ o genoma de cada indivíduo é uma sequência de números reais que indicam os 6 graus de liberdade da molécula do 5-FU ($x,y,z,\theta,\phi,\gamma$). A nossa função de avaliação vai ter que tratar de vários processos:
1. Criar o sistema a partir das moléculas individuais do DNA e 5-FU (transladando e re-orientando a mesma)
1. Chamar o xtb para calcular a energia do sistema.
1. Devolver o valor da energia à função que chama.

A cada passo, vamos também querer guardar a melhor solução, para depois podermos reconstruir a trajectória do melhor complexo 5-FU-DNA.

Também criamos uma função de acasalamento em que os genes são alternadamente selecionados de cada progenitor, e uma função de mutação que permuta as posições de dois alelos no genoma do novo individuo.

In [ ]:
def evaluate_xtb(v, view=False):
  dx, dy, dz, theta, phi, gamma = v # extrair coordenadas de v
  theta = theta%(2*np.pi)
  phi = phi%(2*np.pi)
  gamma = gamma%(2*np.pi)
  # fazemos uma copia do 5fu, e re-orientamos
  new_fu = fu.copy()
  new_fu.orient(dx,dy,dz,theta,phi,gamma)
  # adicionamos a copia do 5fu ao DNA
  system = dna + new_fu
  if view:
    system.show()
  # verificamos colisões
  #if system.check_for_colisions(r=0.8):
  #  return 0.0
  #else:
    # calculamos a energia
  return system.get_gnfff_energy()

def evaluate_mmff94(colv):
  with open('Molecule_mmff94.xyz','w') as f:
    failed=list()
    for iv,v in enumerate(colv):
      dx, dy, dz, theta, phi, gamma = v # extrair coordenadas de v
      theta = theta%(2*np.pi)
      phi = phi%(2*np.pi)
      gamma = gamma%(2*np.pi)
      # fazemos uma copia do 5fu, e re-orientamos
      new_fu = fu.copy()
      new_fu.orient(dx,dy,dz,theta,phi,gamma)
      # adicionamos a copia do 5fu ao DNA
      system = dna + new_fu
      if system.check_for_colisions(r=0.7):
        failed.append(iv)
      f.write(system.write_xyz())
  r=shell("obenergy -ff MMFF94 Molecule_mmff94.xyz | grep 'TOTAL ENERGY ='")
  data=r.decode().split('\n')
  o=np.zeros(len(colv))
  shift=0
  count=0
  for line in data:
    if 'TOTAL ENERGY =' in line:
      if count+shift in failed:
        shift+=1
      o[count+shift]=4.184*float(line.split()[-2])
      count += 1
  return o


def cross(p1,p2):
  # combinação alternada de dois genomas, 
  # com incorporação de 10% de cada um dos progenitores
  new_genome=list()
  for i in range(len(p1)):
    if i%2 == 0:
      new_genome.append(p1[i])
    else:
      new_genome.append(p2[i])
  new_genome = 0.8*np.array(new_genome)+0.1*np.array(p1)+0.1*np.array(p2)
  return ga.Individual(new_genome)

def mutate(p, rng):
  # permuta de dois alelos com introdução de um pouco de ruído
  i1 = rng.integers(len(p))
  i2 = i1
  while(i2==i1):
    i2 = rng.integers(len(p))
  tmp=p[i1]
  p[i1]=p[i2]*rng.normal(1.0,0.05)
  p[i2]=tmp*rng.normal(1.0,0.05)
  return p


Agora temos que decidir os parâmetros do algoritmo genético (GA). Em primeiro lugar temos que decidir a _pool_ de onde vamos gerar os primeiros genomas. No nosso caso, queremos que os angulos possam cobrir o máximo de amplitude de 0 a 360 e que as distância sejam ligeiramente superiores a 30 Å, de forma a podermos cobrir toda uma esfera em torno do nosso fragemnto de DNA. Uma vez que tivemos o cuidado de converter os ângulos para o domínio entre 0 e 2π (ver função acima), podemos estar tranquilos quanto à possibilidade de ter valores demasiado grandes para estes. Assim, a nossa _pool_ é uma amostra de números reais, entre -30 e 30.

As nossas que implementam o algoritmo genético (incluindo acasalamento e mutação) já estão disponíveis no pacote `ga.py`, pelo que resta definir o nº de individuos na população, a fracção de mortes em cada iteração e a probabilidade de ocorrerem mutações.

In [ ]:

n_individuos = 20
n_genes = 6
pool = np.linspace(-30,30,17)
death_ratio = 0.3
mutation_prob = 0.1
n_iterações = 75

start=time.time()
pop = ga.Population(n_individuos, n_genes, pool, 
                    score_func= evaluate_xtb, cross_func= cross, 
                    mut_func= mutate, death_ratio= death_ratio,
                    mut_prob= mutation_prob, collective_eval=False, random_state=42)
print(f"Criar a população inicial demorou {time.time()-start:0.1f} segundos")

Agora, podemos fazer as iterações, e gravar o melhor indivíduo em cada iteração (mais tarde podemos recuperar a geometria correspondente, pelo que assim poupamos memória no computador).

In [ ]:
history=list()
av_scores=list()
sd_scores=list()
history.append(pop[0]) # guardamos o melhor individuo a cada iter.
scores=[i.score for i in pop]
av_scores.append(np.mean(scores)) # média dos scores
sd_scores.append(np.std(scores)) # desvio-padrão dos scores

print(f"Energia da melhor configuração da população inicial: {evaluate_xtb(pop[0],view=True):0.6f}")
start=time.time()
for n in range(n_iterações):
  try:
    pop._iterate()
  except ValueError:
    print("Population Stagnated")
    break
  history.append(pop[0])
  scores=[i.score for i in pop]
  av_scores.append(np.mean(scores))
  sd_scores.append(np.std(scores))
  print(f"Iter {n+1} best_score= {pop[0].score} pop_av= {np.mean(scores)} pop_sd= {np.std(scores)}")
print(f"{n_iterações} iterações do GA demoraram {time.time()-start:0.1f} segundos.")
print(f"Energia da melhor configuração da população final: {evaluate_xtb(pop[0],view=True):0.6f}")


Finalmente, criamos o ficheiro `xyz` com as melhores geometrias a cada iteração, e aproveitamos para fazer uma análise da média da evolução das energias com o nº de iterações.

In [ ]:
with open('ga_history.trj','w') as f:
  for n,v in enumerate(history):
    dx, dy, dz, theta, phi, gamma = v 
    theta = theta%(2*np.pi)
    phi = phi%(2*np.pi)
    gamma = gamma%(2*np.pi)
    new_fu = fu.copy()
    new_fu.orient(dx,dy,dz,theta,phi,gamma)
    system = dna + new_fu
    f.write(system.write_xyz(comment=f"Iter {n+1} energy= {v.score}"))

import pandas as pd
pd.DataFrame({'Iteration':np.arange(len(history)),
              'Best_score':[i.score for i in history],
              'Av_score':av_scores,
              'Std_score':sd_scores}).to_csv('ga_history.csv', index=False)


Finalmente, fazemos um zip (`TP6.zip`) de todos os ficheiros que podemos precisar para a resolução das questões (para além dos dados calculados neste _notebook_).

In [ ]:
! zip TP6.zip 5fu.xyz ACG.xyz ga_history.* 

# Questões

1. Indique a energia-base do sistema DNA+ 10 (5-FU).
1. Como caracteriza o sistema 5-FU-DNA antes no ponto inicial da pesquisa?
1. Como caracteriza o sistema 5-FU-DNA no final da pesquisa?
1. Estime a energia de interacção 5-FU-DNA em kJ/mol.